![](https://i.pinimg.com/564x/e5/2c/d7/e52cd73a24649c74c229cf477b835454.jpg)

# eCompass

![](frames.png)

The [rotation matrix](https://www.mathworks.com/help/fusion/ref/ecompass.html#mw_ecf9e057-074a-4fe7-b63e-0d50d95946a0) is given by:

$$
R = 
\begin{bmatrix}
((a \times m) \times a && a \times m && a
\end{bmatrix}
$$

then normalized across the columns.

## References

- Mathworks: [ecompass](https://www.mathworks.com/help/fusion/ref/ecompass.html#mw_55a75a75-2bfc-4890-9d2d-42e3258c71e5)
- Github: [Open source sensor fusion](https://github.com/memsindustrygroup/Open-Source-Sensor-Fusion)

In [1]:
import numpy as np 
from numpy import cos, sin, pi, sqrt
from numpy import arcsin as asin
from numpy import arccos as acos
from numpy import arctan2 as atan2
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

from scipy import linalg
import sys

import the_collector # read data
print(f"the-collector: {the_collector.__version__}")

from the_collector import BagIt
from the_collector import Pickle, Json

from squaternion import Quaternion

import pandas as pd

%matplotlib inline
from matplotlib import pyplot as plt

# from math import sqrt, atan2, asin, pi
from math import radians as deg2rad
from math import degrees as rad2deg

from slurm import storage

from datetime import datetime
import os

the-collector: 0.8.7


In [2]:
# let's load in some data and have a look at what we have
def bag_info(bag):
    print('Bag keys:')
    print('-'*50)
    for k in bag.keys():
        print(f'  {k:>10}: {len(bag[k]):<7}')

bag = BagIt(Pickle)
fname = "~/github/data-ins-1/2020-5-3-imu/still-z-up.pickle.bag"
fname = os.path.expanduser(fname)

data = bag.read(fname)
bag_info(data)

>> Reading[pickle]: /Users/kevin/github/data-ins-1/2020-5-3-imu/still-z-up.pickle.bag
Bag keys:
--------------------------------------------------
       accel: 2001   
         mag: 2001   
        gyro: 2001   


In [3]:
def normalize3(x, y, z):
    """Return a unit vector"""
    norm = sqrt(x * x + y * y + z * z)

    # already a unit vector
    if norm == 1.0:
        return (x, y, z)

    inorm = 1.0/norm
    if inorm > 1e-6:
        x *= inorm
        y *= inorm
        z *= inorm
    else:
        raise ZeroDivisionError(f'norm({x:.4f}, {y:.4f}, {z:.4f},) = {inorm:.6f}')
    return (x, y, z,)

def ecompass(accel, mag):
    try:
        mx, my, mz = mag
        ax, ay, az = normalize3(*accel)

        pitch = asin(-ax)

        if abs(pitch) >= pi/2:
            roll = 0.0
        else:
            roll = asin(ay/cos(pitch))

        # mx, my, mz = mag
        x = mx*cos(pitch)+mz*sin(pitch)
        y = mx*sin(roll)*sin(pitch)+my*cos(roll)-mz*sin(roll)*cos(pitch)
        heading = atan2(y, x)

        # wrap heading between 0 and 360 degrees
        heading = heading % (2*pi)

        return Quaternion.from_euler(roll, pitch, heading)

    except ZeroDivisionError:
        raise

def ecompass_rot(a,m):
    # https://www.mathworks.com/help/fusion/ref/ecompass.html#mw_ecf9e057-074a-4fe7-b63e-0d50d95946a0
    n = np.cross(a,m)
    m = np.cross(n,a)
    o = a
    
    oo = np.linalg.norm(o)
    if oo < 1e-6:
        raise Exception("eCompass: accel == 0.0")

    m = m/np.linalg.norm(m)
    n = n/np.linalg.norm(n)
    o = o/oo

    r = np.array([m,n,o]).T
    return r

In [4]:
# https://www.mathworks.com/help/fusion/ref/ecompass.html
# m = [19.535 -5.109 47.930];
# a = [0 0 9.8];
# q = ecompass(a,m);
# quaterionEulerAngles = eulerd(q,'ZYX','frame')
# 14.6563         0         0
m = np.array([19.535, -5.109, 47.930])
a = np.array([0, 0, 9.8])
q = ecompass(a,m)
print(q)
print(q.to_euler(degrees=True))
print(np.array(q.to_rot()))

Quaternion(w=-0.9918318627109567, x=0.0, y=0.0, z=0.12755217015525)
(0.0, -0.0, -14.65632877611015)
[[ 0.967  0.253  0.   ]
 [-0.253  0.967  0.   ]
 [ 0.     0.     1.   ]]


In [5]:
# https://www.mathworks.com/help/fusion/ref/ecompass.html#mw_ecf9e057-074a-4fe7-b63e-0d50d95946a0
m = np.cross(np.cross(a,m),a)
n = np.cross(a,m)
o = a

m = m/np.linalg.norm(m)
n = n/np.linalg.norm(n)
o = o/np.linalg.norm(o)

rr = np.array([m,n,o]).T
print(rr)

[[ 0.967  0.253  0.   ]
 [-0.253  0.967  0.   ]
 [ 0.    -0.     1.   ]]


In [6]:
a = np.array([0,0,1])
r = ecompass_rot(a,m)
print(r)
print(np.linalg.det(r))

theta = -asin(r[0,2])
ct = 1/cos(theta)
psi = atan2(r[1,2]*ct,r[2,2]*ct)
rho = atan2(r[0,1]*ct,r[0,0]*ct)
print(f">> {rho*180/pi} {theta*180/pi} {psi*180/pi}")

[[ 0.967  0.253  0.   ]
 [-0.253  0.967  0.   ]
 [ 0.    -0.     1.   ]]
1.0
>> 14.656328776110113 -0.0 0.0


In [7]:
500.789%360.0

140.789

In [8]:
-45.5%360.0

314.5

In [81]:

# https://www.mathworks.com/help/fusion/ref/imufilter-system-object.html
# The imufilter System object™ fuses accelerometer and gyroscope 
# sensor data to estimate device orientation.
def imufilter(sRate, accelN, gyroN):
        """
        FUSE = imufilter returns an indirect Kalman filter System object, 
        FUSE, for fusion of accelerometer and gyroscope data to estimate 
        device orientation. The filter uses a nine-element state vector 
        to track error in the orientation estimate, the gyroscope bias 
        estimate, and the linear acceleration estimate.
        
        sRate: sample rate [Hz]
        gyroN: gryo noise variance [(rad/s)^2]
        accelN: accel noise variance [(m/s^2)^2]
        """
        return KF()

class Fuse:
    """
    X=[vel,pos,orient]"""
    def __init__(self):
        pass
    def step(self):
        pass
    def reset(self):
        pass

In [ ]:
# https://www.mathworks.com/help/fusion/ref/insfiltererrorstate.html
def insfilterErrorState:
    """
    17-state
    X=[q,pos_ned,vel_ned,gyrobias,accelbias,scaleFactor]
    scaleFactor - for pose estimate
    """
    return KF()

# https://www.mathworks.com/help/fusion/ref/insfiltermarg.html
class insfilterMARG:
    """
    22-state
    X=[q,pos_ned,vel_ned,gyrobias,accelbias,magfieldest,magbias]"""